In [1]:
import pandas as pd
import numpy as np
import sys
import jieba
import os
import pickle   #持久化
from numpy import *
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer #TF-IDF向量转换类
from sklearn.feature_extraction.text import TfidfVectorizer #TF_IDF向量生成类
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer 
from sklearn.datasets.base import Bunch
from sklearn.naive_bayes import MultinomialNB #多项式贝叶斯算法
import gc #解決memory error問題
from sklearn.utils import shuffle
import random
import gensim
from gensim.models import word2vec
from gensim.models import doc2vec
import pickle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import np_utils
from tensorflow.keras import Sequential

Using TensorFlow backend.


In [2]:
with open('toutiao_cat_data.txt', 'r', encoding='utf8') as f:
    data = f.read().splitlines()

In [3]:
data

['\ufeff6551700932705387022_!_101_!_news_culture_!_京城最值得你来场文化之旅的博物馆_!_保利集团,马未都,中国科学技术馆,博物馆,新中国',
 '6552368441838272771_!_101_!_news_culture_!_发酵床的垫料种类有哪些？哪种更好？_!_',
 '6552407965343678723_!_101_!_news_culture_!_上联：黄山黄河黄皮肤黄土高原。怎么对下联？_!_',
 '6552332417753940238_!_101_!_news_culture_!_林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？_!_',
 '6552475601595269390_!_101_!_news_culture_!_黄杨木是什么树？_!_',
 '6552387648126714125_!_101_!_news_culture_!_上联：草根登上星光道，怎么对下联？_!_',
 '6552271725814350087_!_101_!_news_culture_!_什么是超写实绘画？_!_',
 '6552452982015787268_!_101_!_news_culture_!_松涛听雨莺婉转，下联？_!_',
 '6552400379030536455_!_101_!_news_culture_!_上联：老子骑牛读书，下联怎么对？_!_',
 '6552339283632455939_!_101_!_news_culture_!_上联：山水醉人何须酒。如何对下联？_!_',
 '6552387367334838792_!_101_!_news_culture_!_国画山水，如何读懂山水画_!_林风眠,黄海归来步步云,秋山图,计白当黑,山水画,江山万里图,张大千,巫峡清秋图,活眼,山雨欲来图',
 '6552314684383429128_!_101_!_news_culture_!_一元硬币是这种，现在价值24000元，找找看！_!_牡丹,收藏价值',
 '6552128476109865229_!_101_!_news_culture_!_有哪些让人感动的语句呢？_!_',
 '6552447749072093453_!_101_!_news_culture_!_上联

In [4]:
label = []
title = []
keyword = []

for i in range(len(data)):
    label.append(data[i].split('_!_')[1])
    title.append(data[i].split('_!_')[3])
    keyword.append(data[i].split('_!_')[4])

In [5]:
keyword = pd.DataFrame(keyword)
title = pd.DataFrame(title)
label = pd.DataFrame(label)
trainData = pd.concat( [title , keyword], axis=1 )
trainData = pd.concat( [trainData , label], axis=1 )
trainData.columns = ['title', 'keyword', 'label']
trainData = shuffle(trainData)

# trainData = trainData[:20000]
trainData.index = range(len(trainData)) # row index從0開始
trainData = trainData.drop(['keyword'], axis=1)
trainData.head()

,title,label
0,联想手机新掌门宣战小米，“新国民旗舰”让全面屏进入3.0时代？,109
1,26岁了，存款有6百多万，是买房投资还是理财比较好？,104
2,五一假期中国游客在韩国疯狂”扫货”，乐天暗喜，销售增长45%，你怎么看？,112
3,任志强预测2018年房地产要降，是真的要降价吗？,106
4,“农家蛋”“土鸡蛋”等比普通鸡蛋价格贵一倍，营养也高么？,115


In [6]:
# #解決memory error問題，釋放記憶體
# del label
del title
del keyword
gc.collect()

70

In [7]:
#停用詞
stopwords=[]
for word in open('stopwords.txt','r', encoding='utf8'):
    stopwords.append(word.strip())

In [8]:
print(sys.getdefaultencoding())

utf-8


In [9]:
title_file = open("title_split.txt", "w", encoding = 'utf8')

#分詞(title和keyword)
splitTitle = []
for i in range(len(trainData.title)):
    split = jieba.cut(trainData.title[i], cut_all=False)
    temp = ""
    for word in split:
        if word not in stopwords:  #若word不在停用詞裡面，則加入
            temp = temp + word + " "
    temp = temp.strip()  #刪除不必要的空白
    
    title_file.writelines(str(temp + "\n"))   #存檔(分割詞，已扣除stopword)
    splitTitle.append(temp)

title_file.close()
    
# splitKeyword = []
# for i in range(len(trainData.keyword)):
#     split = jieba.cut(trainData.keyword[i], cut_all=False)
#     temp = ""
#     for word in split:
#         if word not in stopwords:  #若word不在停用詞裡面，則加入
#             temp = temp + word + " "   
#     temp = temp.strip()  #刪除不必要的空白
#     splitKeyword.append(temp)

Building prefix dict from the default dictionary ...
I0803 18:21:11.820560  6100 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jack\AppData\Local\Temp\jieba.cache
I0803 18:21:11.833568  6100 __init__.py:131] Loading model from cache C:\Users\jack\AppData\Local\Temp\jieba.cache
Loading model cost 0.932 seconds.
I0803 18:21:12.757017  6100 __init__.py:163] Loading model cost 0.932 seconds.
Prefix dict has been built succesfully.
I0803 18:21:12.758022  6100 __init__.py:164] Prefix dict has been built succesfully.


In [10]:
del stopwords
del split
del temp
gc.collect()

197

In [11]:
splitTitle

['联想 手机 新 掌门 宣战 小米 新 国民 旗舰 全面 屏 进入 3.0 时代',
 '26 岁 存款 百 多万 买房 投资 理财 比较',
 '五一 假期 中国 游客 韩国 疯狂 扫货 乐天 暗喜 销售 增长 45%',
 '任志强 预测 2018 年 房地产 要降 真的 降价',
 '农家 蛋 土 鸡蛋 普通 鸡蛋 价格 贵 一倍 营养 高',
 '评价 电影 东成西就',
 '首届 北斗 之星 创新 创业 大赛 开幕',
 '微信 支付宝 正式 宣布 收编 用户 商家 会 影响',
 '儿媳妇 参加 同学聚会 婆婆 亲自 带 打扮 太漂亮',
 '母亲 救 一小 乞丐 回家 乞丐 母亲 养老 钱 抢走 实情 令人感动',
 '迷你 世界 世界 泰拉 瑞亚 生存 战争 蓝图 奶块 抄袭',
 '浙江省 几所 大学 首批 获得 博士学位 授予权',
 '万达 商业 富力 房企 A股 IPO 排序 集体 倒退',
 '户口 本村 本村 土地 户口 迁回',
 '上课时 学生 手机 响个 不停 老师 一怒之下 手机 摔 家长 发票 老师 赔 看待 这种 事',
 '玩 夏侯惇 想半肉 半 输出 想要 全肉 那种 没有 帮忙 1v2 那种',
 '高考 备考 高考 数学 函数 专练 部分 题型 多种 解法 （ 换种 思路 做 题 ）',
 '阿凡 达 开拍 2020 年 上映 前 够 撼动 187 亿 纪录',
 '亚冠 / 决赛 首回合 鹿岛 鹿角 vs 上海 上港   胡 尔克 或缺 战',
 '马云 经典 演讲 永心 听 完 一个 人生',
 '李连杰 古装 剑 甄子丹 古装 剑 成龙 古装 剑 比不上',
 '微信 提现 银行卡 不收 手续费',
 '爱心 助农 悲情 牌',
 '明星 代言 哪家 强 盘点 各大 空调 品牌 代言人',
 '北京 国际 车展 新车 月 河南',
 '看过 美国 狱警 收拾 监狱 犯人 手段 绝对 不想 进去',
 '电视剧 取名 关乎 成败 剧名 含义 绝对 看不懂',
 '龙头 （ 妖股 ） 首阴 战法 … … ~',
 '愿君 心似 我心 下句',
 '四川 平昌 乡镇 入选 四川 十大 最美 茶乡',
 '昆明 破 酥 包子 名字',
 '共享 汽车 真的 共享 经济 一个 风口',
 '准备 买车 朋友 注意 即日起

In [12]:
TaggededDocument = gensim.models.doc2vec.TaggedDocument

In [13]:
# 把splitTitle改變成所需要的輸入格式，gensim裡的Doc2vec的輸入為固定格式 [句子,句子序號] ，需要用TaggedDocument來包裝輸入的句子
train = []
for i, text in enumerate(splitTitle):    # enumerate是把格式變成 [(0, 'Spring'), (1, 'Summer')]
    word_list = text.split(' ')
    l = len(word_list)
    word_list[l-1] = word_list[l-1].strip()
    document = TaggededDocument(word_list, tags = [i])
    train.append(document)

In [14]:
model = doc2vec.Doc2Vec(train, min_count=2, window=5, size=300, sample=1e-3, nagative=5, workers=4)
model.train(train, total_examples=model.corpus_count, epochs=10)

D:\Anaconda\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
W0803 18:24:33.995692  6100 base_any2vec.py:1182] Effective 'alpha' higher than previous training cycles


In [15]:
model

In [16]:
len(model[0])

300

In [17]:
train_arrays = np.zeros((len(trainData), 300))

In [18]:
for i in range(len(trainData)):
    train_arrays[i] = model.infer_vector([splitTitle[i]])

In [19]:
train_arrays.shape

(382688, 300)

In [20]:
# train_arrays[0]

In [21]:
# 對dataset label 做編碼
le = LabelEncoder()
new_label = le.fit_transform(trainData.label).reshape(-1,1)

In [22]:
new_label.shape

(382688, 1)

In [23]:
# label list
new_label = new_label.tolist()

In [24]:
#要先轉乘np.array
new_label = np.array(new_label, dtype='float64')
# label encoding
new_label = np_utils.to_categorical(new_label,15)
print(new_label.shape)

(382688, 15)


In [25]:
new_label[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [26]:
# 随机选择20%作为测试集，剩余作为训练集
X_train, X_test, y_train, y_test = train_test_split(train_arrays, new_label, test_size=0.20, random_state=0)

In [27]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(306150, 300)
(76538, 300)
(306150, 15)
(76538, 15)


In [28]:
model = Sequential()

model.add(Dense(512, input_dim=(X_train.shape[1]) , activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=15,activation='softmax'))

model.summary()
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])

model_fit = model.fit(X_train,y_train,batch_size=100,epochs=10,
                      validation_data=(X_test,y_test),
#                      callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] ## 當val-loss不再提升時停止訓練
                     )

W0803 18:28:19.199299  6100 deprecation.py:506] From D:\Anaconda\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               154112    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                1

In [29]:
test_pred = model.predict(X_test)

In [30]:
test_pred[0]

array([0.01861127, 0.06443188, 0.10559724, 0.10000285, 0.07227954,
       0.04639824, 0.09321243, 0.07228301, 0.10259405, 0.06615007,
       0.05734689, 0.06666736, 0.00151367, 0.05348979, 0.07942165],
      dtype=float32)

In [31]:
result = model.evaluate(X_train, y_train)
print ('\ntrain Acc:', result[1])

306150/306150 [==============================] - 6s 20us/sample - loss: 2.5078 - acc: 0.1314

train Acc: 0.1314421


In [32]:
result = model.evaluate(X_test, y_test)
print ('\ntest Acc:', result[1])

76538/76538 [==============================] - 2s 20us/sample - loss: 2.5102 - acc: 0.1298

test Acc: 0.1297656
